# Notebook to test the `Unsloth` framework with TRL for LLM Fine-tuning

## Install Dependencies

In [1]:
!pip install -qU transformers datasets

In [2]:
import pandas as pd
import json
import pprint

## Fine-tune Example on the IMDB Dataset

In [ ]:
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"


Looking for: ['aiohttp']

rapidsai/linux-64                                           Using cache
rapidsai/noarch                                             Using cache
nvidia/linux-64                                             Using cache
nvidia/noarch                                               Using cache
conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
[+] 0.0s
pkgs/r/noarch                                                 No change
pkgs/main/noarch                                              No change
pkgs/main/linux-64                                            No change
pkgs/r/linux-64                                               No change

Pinned packages:
  - python 3.10.*


Transaction

  Prefix: /opt/conda

  Updating specs:

   - aiohttp


  Package    Version  Build            Channel           Size
───────────────────────────────────────────────────────────────
  Reinstall

In [ ]:
!pip install -q "git+https://github.com/huggingface/transformers.git"

In [ ]:
!pip install -q trl

In [ ]:
from unsloth import FastLanguageModel

## Fine-tune on CodeSearchNet

In [ ]:
from datasets import load_dataset

dataset = load_dataset("code-search-net/code_search_net", "python", trust_remote_code=True)

The repository for code-search-net/code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code-search-net/code_search_net.

You can avoid this prompt in future by passing the argument `trust_remote_code=True`.



Do you wish to run the custom code? [y/N] T

The repository for code-search-net/code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code-search-net/code_search_net.

You can avoid this prompt in future by passing the argument `trust_remote_code=True`.



Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 412178
    })
    test: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 22176
    })
    validation: Dataset({
        features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'],
        num_rows: 23107
    })
})

In [ ]:
dataset["train"][0]['func_code_string']

"def zmq_device(self):\n        '''\n        Multiprocessing target for the zmq queue device\n        '''\n        self.__setup_signals()\n        salt.utils.process.appendproctitle('MWorkerQueue')\n        self.context = zmq.Context(self.opts['worker_threads'])\n        # Prepare the zeromq sockets\n        self.uri = 'tcp://{interface}:{ret_port}'.format(**self.opts)\n        self.clients = self.context.socket(zmq.ROUTER)\n        if self.opts['ipv6'] is True and hasattr(zmq, 'IPV4ONLY'):\n            # IPv6 sockets work for both IPv6 and IPv4 addresses\n            self.clients.setsockopt(zmq.IPV4ONLY, 0)\n        self.clients.setsockopt(zmq.BACKLOG, self.opts.get('zmq_backlog', 1000))\n        self._start_zmq_monitor()\n        self.workers = self.context.socket(zmq.DEALER)\n\n        if self.opts.get('ipc_mode', '') == 'tcp':\n            self.w_uri = 'tcp://127.0.0.1:{0}'.format(\n                self.opts.get('tcp_master_workers', 4515)\n                )\n        else:\n       